In [1]:
import sys
sys.path.insert(0,'../code')

In [2]:
import torch
import torch.nn.functional as F
import numpy as np
import time
from datamodules.transformations import UnNest
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from transformers import ViTFeatureExtractor, ViTForImageClassification
from models.interpretation import ImageInterpretationNet
from torchvision.transforms import Compose
from tqdm.auto import tqdm
from datamodules.image_classification import CIFAR10DataModule
from datamodules.transformations import UnNest
from math import sqrt
from attributions.grad_cam import grad_cam
from attributions.attention_rollout import attention_rollout

/home/angelos/anaconda3/envs/dl2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load model and data

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

vit = ViTForImageClassification.from_pretrained("tanlq/vit-base-patch16-224-in21k-finetuned-cifar10").to(device)

feature_extractor=ViTFeatureExtractor.from_pretrained(
    "tanlq/vit-base-patch16-224-in21k-finetuned-cifar10", return_tensors="pt"
)
feature_extractor = UnNest(feature_extractor)

dm = CIFAR10DataModule(feature_extractor=feature_extractor, batch_size=10)
dm.prepare_data()
dm.setup()
dataloader = iter(dm.test_dataloader())

images = next(dataloader)[0].to(device)

Files already downloaded and verified
Files already downloaded and verified


# Inference Time

In [4]:
start = time.time()
grad_cam(images, vit, True if device=='cuda' else false)
print(f"Inference time for Grad-CAM {time.time() - start}")

Inference time for Grad-CAM 2.5233755111694336


In [5]:
start = time.time()
attention_rollout(images=images, vit=vit, device=device)
print(f"Inference time for Attention Rollout {time.time() - start}")

Inference time for Attention Rollout 0.07604050636291504


In [6]:
diffmask = ImageInterpretationNet.load_from_checkpoint('diffmask.ckpt').to(device)
diffmask.set_vision_transformer(vit)

start = time.time()
diffmask.get_mask(images)["mask"]
print(f"Inference time for DiffMask {time.time() - start}")

Inference time for DiffMask 0.12171101570129395
